# **Final Project Muhammad Bayu Samudra Siddik**

### Import Library

In [ ]:
! pip install surprise
! pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366570 sha256=08a9bd9570450eed45833d577d393346fa1e16166ddd50bd76dc6b5190fcffab
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

from surprise import dataset
from surprise import Reader

from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import BaselineOnly

from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import LeaveOneOut
from surprise.model_selection import KFold
from surprise import accuracy

from collections import defaultdict

import pandas as pd
import numpy as np
import opendatasets as od

from collections import Counter
from scipy import sparse

### Load Dataset

In [ ]:
od.download("https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: bayusiddik
Your Kaggle Key: ··········


100%|██████████| 242M/242M [00:01<00:00, 155MB/s]


In [ ]:
file =('amazon-fine-food-reviews/\
Reviews.csv')
df = pd.read_csv(file)
  
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
df.shape

(568454, 10)

### Preprocessing

#### Part 1 : Create New Data from Existing

**Info Dataset**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


**Create New Dataset from Existing**

In [ ]:
new_df = df[['UserId','ProductId', 'Score']].copy()

In [ ]:
new_df.rename(columns = {'UserId':'userID', 'ProductId':'itemID', 'Score':'rating'}, inplace = True)

In [ ]:
new_df.head()

,userID,itemID,rating
0,A3SGXH7AUHU8GW,B001E4KFG0,5
1,A1D87F6ZCVE5NK,B00813GRG4,1
2,ABXLMWJIXXAIN,B000LQOCH0,4
3,A395BORC6FGVXV,B000UA0QIQ,2
4,A1UQRSCLF8GW1T,B006K2ZZ7K,5


**Check Missing Value**

In [ ]:
new_df.isna().sum() / len(new_df) * 100

userID    0.0
itemID    0.0
rating    0.0
dtype: float64

#### Part 2 : Fitur Encoding

In [ ]:
def encode_column(column):
    keys = column.unique()
    key_to_id = {key:idx for idx,key in enumerate(keys)}
    return key_to_id, np.array([key_to_id[x] for x in column]), len(keys)

In [ ]:
def encode_df(food_df):
    
    food_id, food_df['itemID'], num_food = encode_column(food_df['itemID'])
    user_id, food_df['userID'], num_user = encode_column(food_df['userID'])
    return food_df, num_user, num_food, user_id, food_id

In [ ]:
food_df, num_user, num_food, user_id, food_id = encode_df(new_df)
print("Number of users :", num_user)
print("Number of foods :", num_food)

Number of users : 256059
Number of foods : 74258


**Check Missing Value Again**

In [ ]:
new_df.isna().sum() / len(new_df) * 100

userID    0.0
itemID    0.0
rating    0.0
dtype: float64

**Result New Data**

In [ ]:
food_df

,userID,itemID,rating
0,0,0,5
1,1,1,1
2,2,2,4
3,3,3,2
4,4,4,5
...,...,...,...
568449,256056,74254,5
568450,29529,74255,2
568451,256057,74256,5
568452,163453,74256,5


**Info Result New Data**

In [ ]:
food_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   userID  568454 non-null  int64
 1   itemID  568454 non-null  int64
 2   rating  568454 non-null  int64
dtypes: int64(3)
memory usage: 13.0 MB


**Convert and Save As CSV**

In [ ]:
food_df.to_csv('food_df.csv', encoding='utf-8', index=False)

#### Part 3 : Convert to Reader

**Count Unique Each Value**

In [ ]:
print("Total of unique food : " ,food_df["itemID"].nunique())
print("Total of unique user : " ,food_df["userID"].nunique())
print("Total of unique rating : " ,food_df["rating"].nunique())

Total of unique food :  74258
Total of unique user :  256059
Total of unique rating :  5


**Convert to Reader Dataset**

In [ ]:
min_rating = food_df.rating.min()
max_rating = food_df.rating.max()

In [ ]:
reader = Reader(line_format='user item rating', rating_scale=(min_rating, max_rating))

class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, df, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(food_df['userID'], food_df['itemID'], food_df['rating'])]
        self.reader=reader

data = MyDataset(food_df, reader)

### Split Data and Modeling

#### Part 1 : Modeling Without Cross Validation

**Split 80% data training and 20% data testing**

In [ ]:
trainset, testset = train_test_split(data, test_size=.20)

algo = [SVD(), SVDpp(), NMF(), BaselineOnly()]
benchmark = []
MAE_listed = []
RMSE_listed = []
MSE_listed = []

for algorithm in algo:
    print("Starting:" ,str(algorithm).split(' ')[0].split('.')[-1])
    algorithm.fit(trainset)
    predictions = algorithm.test(testset)

    MAE_listed.append(accuracy.mae(predictions))
    RMSE_listed.append(accuracy.rmse(predictions))
    MSE_listed.append(accuracy.mse(predictions))
    benchmark.append(str(algorithm).split(' ')[0].split('.')[-1])
    print()

Starting: SVD
MAE:  0.7876
RMSE: 1.0807
MSE: 1.1679

Starting: SVDpp
MAE:  0.7517
RMSE: 1.0588
MSE: 1.1210

Starting: NMF
MAE:  0.8154
RMSE: 1.1441
MSE: 1.3090

Starting: BaselineOnly
Estimating biases using als...
MAE:  0.9017
RMSE: 1.1678
MSE: 1.3638



**Result**

In [ ]:
eval_df = pd.DataFrame({"Algoritma": benchmark,"MAE": MAE_listed, "RMSE": RMSE_listed, "MSE": MSE_listed})
eval_df

,Algoritma,MAE,RMSE,MSE
0,SVD,0.787561,1.080697,1.167906
1,SVDpp,0.751728,1.058795,1.121046
2,NMF,0.815407,1.144121,1.309014
3,BaselineOnly,0.901687,1.167803,1.363763


#### Part 2 : CV Each Algorithms

Algorithm performance with various of n-latent factors

##### BaselineOnly

In [ ]:
# You should now be able to use the data object normally.
algo_svd = BaselineOnly()

# Run 5-fold cross-validation and then print results
cross_validate(algo_svd, data, measures=['MAE','RMSE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating MAE, RMSE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.9009  0.9041  0.9007  0.9042  0.9039  0.9027  0.0016  
RMSE (testset)    1.1658  1.1704  1.1681  1.1709  1.1711  1.1693  0.0020  
Fit time          3.05    3.87    3.32    3.26    3.61    3.42    0.28    
Test time         1.64    1.01    1.58    1.63    1.51    1.47    0.24    


{'test_mae': array([0.90094643, 0.9040645 , 0.9006567 , 0.9042035 , 0.9038595 ]),
 'test_rmse': array([1.16576793, 1.17042515, 1.16811794, 1.17087664, 1.17109574]),
 'fit_time': (3.0531158447265625,
  3.8689687252044678,
  3.3186137676239014,
  3.263148069381714,
  3.6072030067443848),
 'test_time': (1.642256736755371,
  1.0081241130828857,
  1.5805723667144775,
  1.6267049312591553,
  1.5149457454681396)}

##### SVD

**10-latent factors**

In [ ]:
# You should now be able to use the data object normally.
algo_svd = SVD(n_factors=10, n_epochs=20)

# Run 5-fold cross-validation and then print results
cross_validate(algo_svd, data, measures=['MAE','RMSE'], cv=5, verbose=True)

Evaluating MAE, RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.8212  0.8244  0.8241  0.8207  0.8201  0.8221  0.0018  
RMSE (testset)    1.1073  1.1114  1.1097  1.1037  1.1045  1.1073  0.0029  
Fit time          9.82    10.02   9.82    9.43    10.11   9.84    0.23    
Test time         1.69    2.43    1.46    1.50    1.35    1.69    0.39    


{'test_mae': array([0.82121857, 0.82444435, 0.82412823, 0.82070844, 0.82008847]),
 'test_rmse': array([1.10725793, 1.11140904, 1.10969781, 1.10374988, 1.10454331]),
 'fit_time': (9.822938680648804,
  10.021152973175049,
  9.81926155090332,
  9.433661699295044,
  10.11054539680481),
 'test_time': (1.694488286972046,
  2.426546096801758,
  1.4648716449737549,
  1.5020370483398438,
  1.3465030193328857)}

**15-latent factors**

In [ ]:
# You should now be able to use the data object normally.
algo_svd = SVD(n_factors=15, n_epochs=20)

# Run 5-fold cross-validation and then print results
cross_validate(algo_svd, data, measures=['MAE','RMSE'], cv=5, verbose=True)

Evaluating MAE, RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.8173  0.8177  0.8181  0.8142  0.8154  0.8165  0.0015  
RMSE (testset)    1.1046  1.1064  1.1031  1.1024  1.1003  1.1034  0.0020  
Fit time          9.89    10.02   10.22   9.21    9.89    9.85    0.34    
Test time         2.47    1.58    1.53    2.61    1.50    1.94    0.49    


{'test_mae': array([0.81730399, 0.81765067, 0.81811062, 0.8142021 , 0.81542293]),
 'test_rmse': array([1.10456175, 1.10639212, 1.10311986, 1.10244018, 1.10032557]),
 'fit_time': (9.886096715927124,
  10.021372318267822,
  10.221832513809204,
  9.214526176452637,
  9.89327096939087),
 'test_time': (2.4655590057373047,
  1.5847172737121582,
  1.5345265865325928,
  2.6108224391937256,
  1.4996764659881592)}

**20-latent factors**

In [ ]:
# You should now be able to use the data object normally.
algo_svd = SVD(n_factors=20, n_epochs=20)

# Run 5-fold cross-validation and then print results
cross_validate(algo_svd, data, measures=['MAE','RMSE'], cv=5, verbose=True)

Evaluating MAE, RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.8115  0.8126  0.8136  0.8104  0.8104  0.8117  0.0012  
RMSE (testset)    1.1005  1.0998  1.1026  1.0970  1.0981  1.0996  0.0019  
Fit time          10.61   9.96    10.78   10.60   10.39   10.47   0.28    
Test time         1.27    1.65    2.31    1.55    1.30    1.62    0.37    


{'test_mae': array([0.81153711, 0.81256833, 0.81358817, 0.81044249, 0.8103982 ]),
 'test_rmse': array([1.10051233, 1.09980971, 1.1026162 , 1.09703288, 1.09810824]),
 'fit_time': (10.60853362083435,
  9.957655906677246,
  10.775288820266724,
  10.599278688430786,
  10.385993480682373),
 'test_time': (1.274317979812622,
  1.6510553359985352,
  2.306246042251587,
  1.5467588901519775,
  1.3012504577636719)}

##### SVD++

**10-latent factors**

In [ ]:
# You should now be able to use the data object normally.
algo_svd = SVDpp(n_factors=10, n_epochs=20)

# Run 5-fold cross-validation and then print results
cross_validate(algo_svd, data, measures=['MAE','RMSE'], cv=5, verbose=True)

Evaluating MAE, RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7602  0.7623  0.7575  0.7650  0.7632  0.7616  0.0026  
RMSE (testset)    1.0642  1.0648  1.0632  1.0713  1.0681  1.0663  0.0030  
Fit time          17.89   18.32   18.62   17.71   17.71   18.05   0.36    
Test time         4.27    3.66    5.44    4.09    4.52    4.40    0.59    


{'test_mae': array([0.7601755 , 0.76225062, 0.75750451, 0.76502323, 0.76324551]),
 'test_rmse': array([1.06417672, 1.06483939, 1.06324188, 1.07125225, 1.0680991 ]),
 'fit_time': (17.88892364501953,
  18.315444231033325,
  18.61646318435669,
  17.707931756973267,
  17.705594539642334),
 'test_time': (4.274425268173218,
  3.664425849914551,
  5.444692373275757,
  4.091663837432861,
  4.52201509475708)}

**15-latent factors**

In [ ]:
# You should now be able to use the data object normally.
algo_svd = SVDpp(n_factors=15, n_epochs=20)

# Run 5-fold cross-validation and then print results
cross_validate(algo_svd, data, measures=['MAE','RMSE'], cv=5, verbose=True)

Evaluating MAE, RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7561  0.7584  0.7581  0.7562  0.7568  0.7571  0.0009  
RMSE (testset)    1.0629  1.0625  1.0646  1.0625  1.0630  1.0631  0.0008  
Fit time          20.25   19.31   20.41   20.41   19.86   20.05   0.42    
Test time         5.05    4.95    4.25    4.67    4.96    4.78    0.30    


{'test_mae': array([0.75606951, 0.7584025 , 0.75805688, 0.75623396, 0.75681887]),
 'test_rmse': array([1.0629495 , 1.0624772 , 1.06462722, 1.0624529 , 1.0630241 ]),
 'fit_time': (20.24809503555298,
  19.312570571899414,
  20.4114248752594,
  20.41402292251587,
  19.860302686691284),
 'test_time': (5.051048994064331,
  4.952303886413574,
  4.2452452182769775,
  4.665574550628662,
  4.963704586029053)}

**20-latent factors**

In [ ]:
# You should now be able to use the data object normally.
algo_svd = SVDpp(n_factors=20, n_epochs=20)

# Run 5-fold cross-validation and then print results
cross_validate(algo_svd, data, measures=['MAE','RMSE'], cv=5, verbose=True)

Evaluating MAE, RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7516  0.7495  0.7548  0.7564  0.7528  0.7530  0.0024  
RMSE (testset)    1.0588  1.0567  1.0631  1.0653  1.0599  1.0607  0.0031  
Fit time          22.46   23.66   23.36   23.30   24.44   23.44   0.64    
Test time         3.75    4.16    5.57    4.21    4.49    4.44    0.61    


{'test_mae': array([0.75162201, 0.74952757, 0.75476661, 0.7563873 , 0.75276126]),
 'test_rmse': array([1.05878035, 1.05671698, 1.0630599 , 1.06528199, 1.05988617]),
 'fit_time': (22.45629906654358,
  23.65826106071472,
  23.355494260787964,
  23.295533418655396,
  24.441720724105835),
 'test_time': (3.748339891433716,
  4.16366720199585,
  5.5676727294921875,
  4.206966161727905,
  4.494403600692749)}

##### NMF

**10-latent factors**

In [ ]:
# You should now be able to use the data object normally.
algo_svd = NMF(n_factors=10, n_epochs=20)

# Run 5-fold cross-validation and then print results
cross_validate(algo_svd, data, measures=['MAE','RMSE'], cv=5, verbose=True)

Evaluating MAE, RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     1.1767  1.1697  1.1727  1.1783  1.1785  1.1752  0.0034  
RMSE (testset)    1.4041  1.3959  1.3981  1.4050  1.4064  1.4019  0.0041  
Fit time          16.50   17.33   17.47   17.06   16.61   16.99   0.38    
Test time         2.24    1.41    1.46    1.93    1.20    1.65    0.38    


{'test_mae': array([1.17668641, 1.16971636, 1.1726547 , 1.1782718 , 1.17849025]),
 'test_rmse': array([1.40413706, 1.39591353, 1.39805322, 1.4050201 , 1.40643069]),
 'fit_time': (16.50354266166687,
  17.32519268989563,
  17.46797776222229,
  17.062549829483032,
  16.612942218780518),
 'test_time': (2.239612579345703,
  1.4132637977600098,
  1.462320327758789,
  1.9275424480438232,
  1.1950640678405762)}

**15-latent factors**

In [ ]:
# You should now be able to use the data object normally.
algo_svd = NMF(n_factors=15, n_epochs=20)

# Run 5-fold cross-validation and then print results
cross_validate(algo_svd, data, measures=['MAE','RMSE'], cv=5, verbose=True)

Evaluating MAE, RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.8698  0.8664  0.8610  0.8675  0.8636  0.8656  0.0030  
RMSE (testset)    1.1773  1.1745  1.1716  1.1767  1.1682  1.1737  0.0034  
Fit time          19.10   18.88   19.90   18.90   19.07   19.17   0.38    
Test time         1.34    1.48    1.35    2.62    1.80    1.72    0.48    


{'test_mae': array([0.86975975, 0.8664062 , 0.8610225 , 0.86745974, 0.8635851 ]),
 'test_rmse': array([1.17725585, 1.17453196, 1.1716367 , 1.17667671, 1.16817812]),
 'fit_time': (19.099560022354126,
  18.880439043045044,
  19.903969287872314,
  18.898622035980225,
  19.073710680007935),
 'test_time': (1.3390612602233887,
  1.4789142608642578,
  1.3547444343566895,
  2.6150949001312256,
  1.8001065254211426)}

**20-latent factors**

In [ ]:
# You should now be able to use the data object normally.
algo_svd = NMF(n_factors=20, n_epochs=20)

# Run 5-fold cross-validation and then print results
cross_validate(algo_svd, data, measures=['MAE','RMSE'], cv=5, verbose=True)

Evaluating MAE, RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7302  0.7311  0.7320  0.7325  0.7295  0.7311  0.0011  
RMSE (testset)    1.1208  1.1191  1.1224  1.1231  1.1169  1.1205  0.0023  
Fit time          22.21   22.78   22.66   23.48   21.75   22.58   0.58    
Test time         2.21    1.30    1.38    1.32    1.84    1.61    0.36    


{'test_mae': array([0.73018548, 0.73112803, 0.73203248, 0.73248327, 0.72947123]),
 'test_rmse': array([1.12078889, 1.11914394, 1.12243038, 1.1231008 , 1.11685605]),
 'fit_time': (22.207026481628418,
  22.783040761947632,
  22.66493582725525,
  23.48246955871582,
  21.746458292007446),
 'test_time': (2.2104716300964355,
  1.2983524799346924,
  1.3811686038970947,
  1.3208885192871094,
  1.8438341617584229)}